<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/performance-evaluation/funds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoofinancials
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

In [7]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr
import statsmodels.api as sm
from scipy.stats import ttest_1samp as ttest

# ff_monthly data
ff3 = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[0]/100
ff5 = pdr('F-F_Research_Data_5_Factors_2x3','famafrench', start=1900)[0]/100
Mom = pdr('F-F_Momentum_Factor','famafrench', start=1900)[0]/100
ST_Rev = pdr('F-F_ST_Reversal_Factor','famafrench', start=1900)[0]/100
LT_Rev = pdr('F-F_LT_Reversal_Factor','famafrench', start=1900)[0]/100
ff48 = pdr("48_Industry_Portfolios", "famafrench", start=1900)[0]/100

facts = pd.concat((ff5, Mom, ST_Rev, LT_Rev), axis=1).dropna()

In [8]:
# Example
ticker = 'Meta'
tick = 'Meta-RF'
TICKER = pdr(ticker, 'yahoo', start=1970)
TICKER = TICKER['Adj Close'].resample('M').last().pct_change().dropna()
TICKER.name = ticker
TICKER.index = TICKER.index.to_period('M')
ret = TICKER

'''etfs: 'SPY = S&P 500 ETF', 'IVE = S&P 500 Value ETF', 'IVW = S&P 500 Growth ETF', 
'IWB = Russell 1000 ETF', 'IWD = Russell 1000 Value ETF', 'IWF = Russell 1000 Growth ETF', 
'IWM = Russell 2000 ETF', 'IWN = Russell 2000 Value ETF', 'IWO = Russell 2000 Growth ETF', 
IWV = Russell 3000 ETF'
'''
benchmark = pdr('SPY', 'yahoo', start=1970)
benchmark = benchmark['Adj Close'].resample('M').last().pct_change().dropna()
benchmark.name = 'SPY'
benchmark.index = benchmark.index.to_period('M')

df = pd.concat((ret, benchmark, facts), axis=1).dropna()
df['Meta-RF'] = df.Meta - df.RF
df['SPY-RF'] = df.SPY - df.RF
df.head()

,Meta,SPY,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom,ST_Rev,LT_Rev,Meta-RF,SPY-RF
Date,,,,,,,,,,,,,
2012-06,0.050676,0.040581,0.0389,0.0081,0.0059,-0.0107,0.0044,0.0000,-0.0106,-0.0079,0.0166,0.050676,0.040581
2012-07,-0.301929,0.011830,0.0079,-0.0276,-0.0012,0.0113,0.0003,0.0000,0.0302,0.0148,-0.0210,-0.301929,0.011830
2012-08,-0.168125,0.025053,0.0255,0.0045,0.0130,-0.0133,-0.0088,0.0001,-0.0237,0.0303,0.0187,-0.168225,0.024953
2012-09,0.199336,0.025351,0.0273,0.0064,0.0158,-0.0151,0.0151,0.0001,-0.0114,-0.0019,0.0179,0.199236,0.025251
2012-10,-0.025392,-0.018198,-0.0176,-0.0091,0.0356,-0.0134,0.0255,0.0001,0.0014,0.0025,0.0106,-0.025492,-0.018298


In [9]:
# Simple Benchmark
# active return = return - benchmark return
# std of the active return: tracking error
# information ratio = mean / std
xrets = 100 * 100 * (df['Meta'] - df['SPY'])    # basis points per month
raw_mn = xrets.mean()
raw_sd = xrets.std()
raw_info = raw_mn / raw_sd
raw_t, raw_p = ttest(xrets, 0)

tbl = pd.DataFrame(index=[0], columns=['mean', 'std err', 't stat', 'p value', 'info ratio'])
for stat, col in zip([raw_mn, raw_mn/raw_t, raw_t, raw_p], ['mean', 'std err', 't stat', 'p value']):
    tbl[col] = f'{stat:.2f}'
tbl['info ratio'] = f'{raw_info:.2%}'

contribs = pd.DataFrame(dtype=float, index=df.index, columns=['Active', 'Benchmark'])
contribs['Benchmark'] = df['SPY']
contribs['Active'] = df['Meta'] - df['SPY']
cum = (1 + contribs).cumprod()

print(tbl)
print('In the simple benchmark, the active return of Meta in May, 2022 is {:.2%}'.format(cum['Active']['2022-05']/100))
print('In the simple benchmark, the benchmark return of Meta in May, 2022 is {:.2%}'.format(cum['Benchmark']['2022-05']/100))

    mean std err t stat p value info ratio
0  91.51   86.01   1.06    0.29      9.71%
In the simple benchmark, the active return of Meta in May, 2022 is 1.77%
In the simple benchmark, the benchmark return of Meta in May, 2022 is 3.80%


In [10]:
# Beta-adjusted Benchmark
# active return = return in excess of the beta-adjusted benchmark return
result = sm.OLS(df[tick], sm.add_constant(df['SPY-RF'])).fit()
tbl = (result.summary2().tables[1])
tbl = tbl[tbl.columns[:-2]]
tbl.columns = ['coef', 'std err', 't stat', 'p value']
tbl.index = ['alpha', 'SPY-RF']
tbl.loc['alpha', 'coef'] = 100 * 100 * tbl.loc['alpha', 'coef']
tbl.loc['alpha', 'std err'] = 100 * 100 * tbl.loc['alpha', 'std err']
tbl = tbl.round(2)
tbl['info ratio'] = ''
info = result.params['const'] / np.sqrt(result.mse_resid)
tbl.loc['alpha', 'info ratio'] = f'{info:.2%}'
print(tbl)
print('In beta-adjusted benchmark, the monthly benchmark excess return in February, 2022 is {:.2%}'.format(df['SPY-RF']['2022-02']))
print('In beta-adjusted benchmark, the monthly excess return in February, 2022 is {:.2%}'.format(df['Meta-RF']['2022-02']))

         coef  std err  t stat  p value info ratio
alpha   68.19    89.81    0.76     0.45      7.23%
SPY-RF   1.20     0.22    5.39     0.00           
In beta-adjusted benchmark, the monthly benchmark excess return in February, 2022 is -2.95%
In beta-adjusted benchmark, the monthly excess return in February, 2022 is -32.63%


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
